In [1]:
#imports
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim.lr_scheduler as lr_scheduler
from torch.nn import Parameter
import math

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters 
num_epochs = 15
batch_size = 100
learning_rate = 0.01

In [4]:
#Data loading
TRAIN_DIRECTORY = 'E:/CSE465 project/Jupyter Notebooks/CSE465 project (Alphabet)/BSL Alphabet Split/train'
VAL_DIRECTORY = 'E:/CSE465 project/Jupyter Notebooks/CSE465 project (Alphabet)/BSL Alphabet Split/val'
TEST_DIRECTORY = 'E:/CSE465 project/Jupyter Notebooks/CSE465 project (Alphabet)/BSL Alphabet Split/test'

train_data = ImageFolder(TRAIN_DIRECTORY, transform=ToTensor())
val_data = ImageFolder(VAL_DIRECTORY, transform=ToTensor())
test_data = ImageFolder(TEST_DIRECTORY, transform=ToTensor())

train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_dl = DataLoader(val_data, batch_size*2, num_workers=4, pin_memory=True)
val_dl = DataLoader(test_data, batch_size*2,shuffle=True, num_workers=4, pin_memory=True)

In [5]:
#load pretrained model
model = torchvision.models.vgg16_bn(pretrained = True)

In [6]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

In [7]:
for param in model.parameters():
    param.requires_grad = True

model.avgpool = Identity()

model.classifier = nn.Sequential(nn.Linear(in_features=512*3*3, out_features=256, bias=True),
                                 nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(in_features=256, out_features=128, bias=True),
                                 nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True),
                                 nn.ReLU(inplace=True))


model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [8]:
def l2_norm(input, axis = 1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output

In [9]:
class Arcface(nn.Module):
    def __init__(self, embedding_size=128, classnum=35,  s=2, m=0.5):
        super(Arcface, self).__init__()
        self.classnum = classnum
        self.kernel = Parameter(torch.FloatTensor(embedding_size,classnum), requires_grad=True)
        # initial kernel
        self.kernel.data.uniform_(-1, 1).renorm_(2,1,1e-5).mul_(1e5)
        self.m = m # the margin value, default is 0.5
        self.s = s # scalar value default is 64, see normface https://a...content-available-to-author-only...v.org/abs/1704.06369
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
    def forward(self, embbedings, label):
        # weights norm
        nB = len(embbedings)
        kernel_norm = l2_norm(self.kernel,axis=0)
        # cos(theta) using dot product
        cos_theta = torch.mm(embbedings,kernel_norm)
        cos_theta = cos_theta.clamp(-1,1) # for numerical stability
        #Sqrt(1 - cos^2(theta))
        sin_theta = torch.sqrt(torch.clamp((1.0 - torch.pow(cos_theta, 2)),1e-9,1))
        #cos(theta+m) = cos(theta)*cos(m) - sin(theta)*sin(m)
        cos_theta_m = (cos_theta * self.cos_m - sin_theta * self.sin_m)
        output = cos_theta * 1.0 # a little bit hacky way to prevent in_place operation on cos_theta
        idx_ = torch.arange(0, nB, dtype=torch.long)
        output[idx_, label] = cos_theta_m[idx_, label] 
        output *= self.s #scale up in order to make softmax work, first introduced in normface
        return output

In [10]:
metric_fc = Arcface().to(device)
criterion = torch.nn.CrossEntropyLoss()

# optimzer nn
optimizer_nn = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.005)

# optimzer arcface
optimzer_arcface = torch.optim.SGD(metric_fc.parameters(), lr=0.01)

In [11]:
n_total_steps = len(train_dl)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dl):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        
        # set opt
        optimizer_nn.zero_grad()
        optimzer_arcface.zero_grad()

        # Forward pass
        outputs = model(images.float())
        outputs = metric_fc(outputs, labels)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer_nn.step()
        optimzer_arcface.step()
        
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

        if (i+1) % 57 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/15], Step [57/191], Loss: 3.3710
Epoch [1/15], Step [114/191], Loss: 2.8659
Epoch [1/15], Step [171/191], Loss: 2.2401
Epoch [2/15], Step [57/191], Loss: 1.6345
Epoch [2/15], Step [114/191], Loss: 1.3939
Epoch [2/15], Step [171/191], Loss: 1.2168
Epoch [3/15], Step [57/191], Loss: 1.0691
Epoch [3/15], Step [114/191], Loss: 0.9311
Epoch [3/15], Step [171/191], Loss: 0.9253
Epoch [4/15], Step [57/191], Loss: 0.8819
Epoch [4/15], Step [114/191], Loss: 0.8160
Epoch [4/15], Step [171/191], Loss: 0.8075
Epoch [5/15], Step [57/191], Loss: 0.7822
Epoch [5/15], Step [114/191], Loss: 0.8124
Epoch [5/15], Step [171/191], Loss: 0.7596
Epoch [6/15], Step [57/191], Loss: 0.7447
Epoch [6/15], Step [114/191], Loss: 0.7729
Epoch [6/15], Step [171/191], Loss: 0.7403
Epoch [7/15], Step [57/191], Loss: 0.7273
Epoch [7/15], Step [114/191], Loss: 0.7461
Epoch [7/15], Step [171/191], Loss: 0.7339
Epoch [8/15], Step [57/191], Loss: 0.7263
Epoch [8/15], Step [114/191], Loss: 0.7257
Epoch [8/15], Step 

In [12]:
def test(model, metric_fc, criterion, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            features3d = model(data.float())
            
            output = metric_fc(features3d, labels)
            test_loss += criterion(output, labels)
            # test_loss += centerLoss(output, target, device, features3d)
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nResult: Average loss: {}, Accuracy: {}/{} ({}%)\n'.format(
        str(test_loss), str(correct), str(len(test_loader.dataset)),
        str(100. * correct / len(test_loader.dataset))))

In [13]:
test(model, metric_fc, criterion, device, train_dl)


Result: Average loss: tensor(0.0072, device='cuda:0'), Accuracy: 19060/19075 (99.92136304062909%)



In [14]:
test(model, metric_fc, criterion, device, val_dl)


Result: Average loss: tensor(0.0040, device='cuda:0'), Accuracy: 2403/2416 (99.46192052980132%)



In [15]:
test(model, metric_fc, criterion, device, test_dl)


Result: Average loss: tensor(0.0037, device='cuda:0'), Accuracy: 2363/2373 (99.57859249894648%)

